# Guided Capstone Step 4. Pre-Processing and Training Data Development

**The Data Science Method**  


1.   Problem Identification 


2.   Data Wrangling 
  
 
3.   Exploratory Data Analysis   

4.   **Pre-processing and Training Data Development**  
 * Create dummy or indicator features for categorical variables
  * Standardize the magnitude of numeric features
  * Split into testing and training datasets
  * Apply scaler to the testing set
5.   Modeling 
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

**<font color='teal'> Start by loading the necessary packages as we did in step 3 and printing out our current working directory just to confirm we are in the correct project directory. </font>**

In [27]:
#load python packages
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt

In [28]:
print(os.getcwd())

path = os.getcwd()

/Users/lokin/Desktop/Step 2


**<font color='teal'>  Load the csv file you created in step 3, remember it should be saved inside your data subfolder and print the first five rows.</font>**

In [81]:
df = pd.read_csv(path+'/data/step3_output.csv', index_col = 0)

In [82]:
df.head()

,Name,state,summit_elev,vertical_drop,trams,fastEight,fastSixes,fastQuads,quad,triple,...,SkiableTerrain_ac,Snow Making_ac,daysOpenLastYear,yearsOpen,averageSnowfall,AdultWeekday,AdultWeekend,projectedDaysOpen,NightSkiing_ac,clusters
2,Hilltop Ski Area,Alaska,2090,294,0,0.0,0,0,0,1,...,30.0,30.000000,150.000000,36.0,69.0,30.0,34.0,152.0,30.000000,0
4,Sunrise Park Resort,Arizona,11100,1800,0,0.0,0,1,2,3,...,800.0,80.000000,115.000000,49.0,250.0,74.0,78.0,104.0,80.000000,1
5,Yosemite Ski & Snowboard Area,California,7800,600,0,0.0,0,0,0,1,...,88.0,174.873239,110.000000,84.0,300.0,47.0,47.0,107.0,100.395722,1
9,Dodge Ridge,California,8200,1600,0,0.0,0,0,1,2,...,862.0,174.873239,115.103943,69.0,350.0,78.0,78.0,140.0,100.395722,1
10,Donner Ski Ranch,California,8012,750,0,0.0,0,0,0,1,...,505.0,60.000000,163.000000,82.0,400.0,75.0,75.0,170.0,100.395722,1


In [83]:
df_remove_state = df.drop(['state'], axis = 1)

In [54]:
df['state']

2          Alaska
4         Arizona
5      California
9      California
10     California
          ...    
323       Wyoming
326       Wyoming
327       Wyoming
328       Wyoming
329       Wyoming
Name: state, Length: 172, dtype: object

## Create dummy features for categorical variables

**<font color='teal'> Create dummy variables for `state`. Add the dummies back to the dataframe and remove the original column for `state`. </font>**

Hint: you can see an example of how to execute this in Aiden's article on preprocessing [here](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-pre-processing-and-training-data-development-fd2d75182967). 

In [57]:
# dfo = df.select_dtypes(include=['object']) # select object type columns
# df = pd.concat([df.drop(dfo, axis=1), pd.get_dummies(dfo)], axis=1)
state_dummy = pd.get_dummies(df.state)

df = pd.concat([df.drop(['state'], axis=1), state_dummy], axis=1)
# df = df.drop([0])

In [58]:
df.head()

,Name,summit_elev,vertical_drop,trams,fastEight,fastSixes,fastQuads,quad,triple,double,...,Rhode Island,South Dakota,Tennessee,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2,Hilltop Ski Area,2090,294,0,0.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,Sunrise Park Resort,11100,1800,0,0.0,0,1,2,3,1,...,0,0,0,0,0,0,0,0,0,0
5,Yosemite Ski & Snowboard Area,7800,600,0,0.0,0,0,0,1,3,...,0,0,0,0,0,0,0,0,0,0
9,Dodge Ridge,8200,1600,0,0.0,0,0,1,2,5,...,0,0,0,0,0,0,0,0,0,0
10,Donner Ski Ranch,8012,750,0,0.0,0,0,0,1,5,...,0,0,0,0,0,0,0,0,0,0


## Standardize the magnitude of numeric features

**<font color='teal'> Using sklearn preprocessing standardize the scale of the features of the dataframe except the name of the resort which we done't need in the dataframe for modeling, so it can be droppped here as well. Also, we want to hold out our response variable(s) so we can have their true values available for model performance review. Let's set `AdultWeekend` to the y variable as our response for scaling and modeling. Later we will go back and consider the `AdultWeekday`, `dayOpenLastYear`, and `projectedDaysOpen`. For now leave them in the development dataframe. </font>**

In [60]:
# first we import the preprocessing package from the sklearn library
from sklearn import preprocessing

# Declare an explanatory variable, called X,and assign it the result of dropping 'Name' and 'AdultWeekend' from the df
X = df.drop(['Name','AdultWeekend'], axis=1)

# Declare a response variable, called y, and assign it the AdultWeekend column of the df 
y = df.AdultWeekend

# Here we use the StandardScaler() method of the preprocessing package, and then call the fit() method with parameter X 
scaler = preprocessing.StandardScaler().fit(X)

# Declare a variable called X_scaled, and assign it the result of calling the transform() method with parameter X 
X_scaled=scaler.transform(X) 

## Split into training and testing datasets

**<font color='teal'> Using sklearn model selection import train_test_split, and create a 75/25 split with the y = `AdultWeekend`. We will start by using the adult weekend ticket price as our response variable for modeling.</font>**

In [64]:
# Import the train_test_split function from the sklearn.model_selection utility.  
from sklearn.model_selection import train_test_split

# Get the 1-dimensional flattened array of our response variable y by calling the ravel() function on y
y = y.ravel()

# Call the train_test_split() function with the first two parameters set to X_scaled and y 
# Declare four variables, X_train, X_test, y_train and y_test separated by commas 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

Here we start the actual modeling work. First let's fit a multiple linear regression model to predict the `AdultWeekend` price.

# Guided Capstone Step 5. Modeling


This is the fifth step in the Data Science Method. In the previous steps you cleaned and prepared the datasets. Now it's time to get into the most exciting part: modeling! In this exercise, you'll build three different models and compare each model's performance. In the end, you'll choose the best model for demonstrating insights to Big Mountain management.



### **The Data Science Method**  


1.   Problem Identification 

2.   Data Wrangling 
  
3.   Exploratory Data Analysis 
 
4.   Pre-processing and Training Data Development

5.   **Modeling**
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

## Fit Models with a Training Dataset

**<font color='teal'> Using sklearn, fit the model on your training dataset.</font>**

#### Model 1

In [66]:
#all first model set
from sklearn import linear_model
from sklearn.metrics import explained_variance_score,mean_absolute_error

lm = linear_model.LinearRegression()
model = lm.fit(X_train,y_train)

**<font color='teal'> Predict on the testing dataset and score the model performance with the y_test set and the y-pred values. The explained variance is a measure of the variation explained by the model. This is also known as the R-squared value. </font>**

Hint: you will have to use the `predict()` method here as it's used in this [DSM article](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-modeling-56b4233cad1b) about modeling. 

In [67]:
# Make a variable called y_pred and assign it the result of calling predict() on our model variable with parameter X_test
y_pred = model.predict(X_test)

## Review Model Outcomes — Iterate over additional models as needed

In [0]:
# You might want to use the explained_variance_score() and mean_absolute_error() metrics.
# To do so, you will need to import them from sklearn.metrics. 
# You can plug y_test and y_pred into the functions to evaluate the model

In [68]:
from sklearn.mX_tesetrics import explained_variance_score, mean_absolute_error

var = explained_variance_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

In [70]:
print(var)
print(mae)

0.7204613350979476
6.500867562313444


**<font color='teal'> Print the intercept value from the linear model. </font>**

Hint: our linear regression model `lm` has an attribute `intercept_` for the intercept

In [69]:
lm.intercept_

20.317340782756048

**<font color='teal'> The intercept is the mean `AdultWeekend` price for all the resorts given the other characteristics. The addition or subtraction of each of the coefficient values in the regression are numeric adjustments applied to the intercept to provide a particular observation's value for the resulting `AdultWeekend` value. Also, because we took the time to scale our x values in the training data, we can compare each of the coeeficients for the features to determine the feature importances. Print the coefficient values from the linear model and sort in descending order to identify the top ten most important features.</font>** 


Hint: make sure to review the absolute value of the coefficients, because the adjustment may be positive or negative, but what we are looking for is the magnitude of impact on our response variable.

In [77]:
# You might want to make a pandas DataFrame displaying the coefficients for each state like so: 

coeff = pd.DataFrame(abs(lm.coef_), X.columns, columns=['Coefficient'])
print(coeff.sort_values(by=['Coefficient'], ascending=False).head(10))

                Coefficient
New Jersey        16.481126
Tennessee         14.288681
Rhode Island      13.789492
Montana           12.138501
North Carolina    12.071933
South Dakota      10.675081
Wyoming           10.266487
Indiana           10.195237
Massachusetts      9.803482
Idaho              9.709396


**<font color='teal'>You should see that the top ten important features are different states. However, the state is not something the managers at the Big Mountain Resort can do anything about. Given that we care more about actionable traits associated with ticket pricing, rebuild the model without the state features and compare the results. </font>**

Hint: Try to construct another model using exactly the steps we followed above. 

#### Model 2

In [90]:
# first we import the preprocessing package from the sklearn library
from sklearn import preprocessing

# Declare an explanatory variable, called X,and assign it the result of dropping 'Name' and 'AdultWeekend' from the df
X_rs = df_remove_state.drop(['Name','AdultWeekend'], axis=1)

# Declare a response variable, called y, and assign it the AdultWeekend column of the df 
y_rs = df_remove_state.AdultWeekend

# Here we use the StandardScaler() method of the preprocessing package, and then call the fit() method with parameter X 
scaler = preprocessing.StandardScaler().fit(X_rs)

# Declare a variable called X_scaled, and assign it the result of calling the transform() method with parameter X 
X_scaled=scaler.transform(X_rs) 

In [91]:
# Import the train_test_split function from the sklearn.model_selection utility.  
from sklearn.model_selection import train_test_split

# Get the 1-dimensional flattened array of our response variable y by calling the ravel() function on y
y_rs = y_rs.ravel()

# Call the train_test_split() function with the first two parameters set to X_scaled and y 
# Declare four variables, X_train, X_test, y_train and y_test separated by commas 
X_train_rs, X_test_rs, y_train_rs, y_test_rs = train_test_split(X_rs, y_rs, test_size=0.25, random_state=1)

In [92]:
from sklearn.ensemble import RandomForestRegressor

In [93]:
regressor = RandomForestRegressor(random_state=0, n_estimators=200)
regressor.fit(X_train_rs, y_train_rs)
regressor.score(X_test_rs, y_test_rs)

0.6610935665618396

In [96]:
y_pred_rs = regressor.predict(X_test_rs)

In [98]:
from sklearn.metrics import explained_variance_score, mean_absolute_error

var_rs = explained_variance_score(y_test_rs, y_pred_rs)
mae_rs = mean_absolute_error(y_test_rs, y_pred_rs)

In [99]:
print(var_rs)
print(mae_rs)

0.6710972865175432
6.651655355505548


In [107]:
features, importance = [], []
for feat, imp in zip(X.columns, regressor.feature_importances_):-
    features.append(feat)
    importance.append(round(imp, 4))
    
pd.DataFrame({"features":features, "importance": importance}).sort_values(by=['importance'], ascending = False)

,features,importance
19,AdultWeekday,0.6950
11,Runs,0.0343
0,summit_elev,0.0311
17,yearsOpen,0.0267
18,averageSnowfall,0.0245
22,clusters,0.0238
15,Snow Making_ac,0.0209
14,SkiableTerrain_ac,0.0201
1,vertical_drop,0.0200
13,LongestRun_mi,0.0183


**<font color='teal'> When reviewing our new model coefficients, we see `summit_elev` is now in the number two spot. This is also difficult to change from a management prespective and highly correlated with `base_elev` and `vertical_drop`.  This time, rebuild the model without the state features and without the `summit_elev` and without `base_elev`and compare the results. </font>**

#### Model 3

In [113]:
# first we import the preprocessing package from the sklearn library
from sklearn import preprocessing

# Declare an explanatory variable, called X,and assign it the result of dropping 'Name' and 'AdultWeekend' from the df
X_rs = df_remove_state.drop(['Name','AdultWeekend','summit_elev'], axis=1)

# Declare a response variable, called y, and assign it the AdultWeekend column of the df 
y_rs = df_remove_state.AdultWeekend

# Here we use the StandardScaler() method of the preprocessing package, and then call the fit() method with parameter X 
scaler = preprocessing.StandardScaler().fit(X_rs)

# Declare a variable called X_scaled, and assign it the result of calling the transform() method with parameter X 
X_scaled=scaler.transform(X_rs)  

In [114]:
# Import the train_test_split function from the sklearn.model_selection utility.  
from sklearn.model_selection import train_test_split

# Get the 1-dimensional flattened array of our response variable y by calling the ravel() function on y
y_rs = y_rs.ravel()

# Call the train_test_split() function with the first two parameters set to X_scaled and y 
# Declare four variables, X_train, X_test, y_train and y_test separated by commas 
X_train_rs, X_test_rs, y_train_rs, y_test_rs = train_test_split(X_rs, y_rs, test_size=0.25, random_state=1)

In [115]:
from sklearn.linear_model import Ridge

In [116]:
clf = Ridge(alpha=1.0)
clf.fit(X_train_rs, y_train_rs)
clf.score(X_test_rs, y_test_rs)

0.7459029260687654

In [121]:
y_pred_rs = clf.predict(X_test_rs)

In [122]:
from sklearn.metrics import explained_variance_score, mean_absolute_error

var_rs = explained_variance_score(y_test_rs, y_pred_rs)
mae_rs = mean_absolute_error(y_test_rs, y_pred_rs)

In [123]:
print(var_rs)
print(mae_rs)

0.7636649790412824
6.378136875127223


In [119]:
coeff = pd.DataFrame(abs(clf.coef_),X_train_rs.columns, columns=['Coefficient'])
print(coeff.sort_values(by=['Coefficient'], ascending=False).head(10))

               Coefficient
clusters          3.378479
surface           1.260780
quad              1.184372
total_chairs      1.011419
LongestRun_mi     0.870056
fastQuads         0.731300
triple            0.608038
AdultWeekday      0.592023
TerrainParks      0.566091
double            0.251511


## Identify the Final Model

**<font color='teal'> Review the model performances in the table below and choose the best model for proving insights to Big Mountain management about what features are driving ski resort lift ticket prices. Type your choice in the final markdown cell — you will discuss this selection more in the next step of the guided casptone. </font>**

| Model | Explained Variance| Mean Absolute Error|Features Dropped|
| --- | --- | --- | --- |
| Model 1. | 0.72 | 6.5 |-|
| Model 2. | 0.67| 6.65|'state'|
| Model 3. | 0.76 | 6.37|'state','summit_elev','base_elev'|

Model Selection:

Model 3 seems to perform well with higher explained variance score and lower mean absolute error. It is because I have removed the features which are highly influential for model but irrelevant. Therefore, the features in the model 3 are good enough to get reasonable predictions. 